In [ ]:
#XGBoost
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn import metrics
from pandas import read_csv
import xgboost as xgb
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from matplotlib.pyplot import savefig
test_size = 0.1
filename = 'D:\Jupyter notebook\锰基除重金属\独热编码删除所有空缺值.csv'
data0 = read_csv(filename)

# 假设 'data0' 是你的 DataFrame
X = data0.drop(columns=['Adsorption capacity'])  # 选择除了最后一列之外的所有列作为特征
y = data0['Adsorption capacity']   # 选择最后一列作为目标变量

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=6)
scaler = StandardScaler()
scaler.fit(X_train)
X_train_std = scaler.transform(X_train)
X_test_std = scaler.transform(X_test)

param_grid = {'learning_rate': [0.1,0.15,0.35],
              'max_depth': [3],
              'min_child_weight': [3,4,5],
              'colsample_bytree': [0.8],
              'gamma': [0, 0.1, 0.2, 0.3],
              'reg_alpha': [0,0.1,0.2,0.3],
              'reg_lambda': [1,3,5]
              }



param_grid = param_grid
n_splits = 10
seed = 10
k_fold = KFold(n_splits=n_splits, random_state=seed, shuffle=True)
xgb_regressor = xgb.sklearn.XGBRegressor()
model = GridSearchCV(estimator=xgb_regressor, scoring='neg_root_mean_squared_error', param_grid=param_grid, cv=k_fold,
                     verbose=3, n_jobs=-1, return_train_score=True)
model.fit(X_train_std, y_train)
print(model.cv_results_)
df_result = pd.DataFrame(model.cv_results_)
df_result.to_csv()
# fit and predict
y_train_hat = model.predict(X_train_std)
y_test_hat = model.predict(X_test_std)
# plot
fontsize = 12
plt.figure(figsize=(3.5, 3))
plt.style.use('default')
plt.rc('xtick', labelsize=fontsize)
plt.rc('ytick', labelsize=fontsize)
plt.rcParams['font.family'] = "Arial"
a = plt.scatter(y_train, y_train_hat, s=25, c='#b2df8a')
plt.plot([y_train.min(), y_train.max()], [y_train.min(), y_train.max()], 'k:', lw=1.5)
plt.xlabel('Observation', fontsize=fontsize)
plt.ylabel('Prediction', fontsize=fontsize)
plt.tick_params(direction='in')
plt.title(('Train RMSE: {:.2e}'.format(np.sqrt(metrics.mean_squared_error(y_train, y_train_hat))),
           'Test RMSE: {:.2e}'.format(np.sqrt(metrics.mean_squared_error(y_test, y_test_hat)))), fontsize=fontsize)
b = plt.scatter(y_test, y_test_hat, s=25, c='#1f78b4')
plt.legend((a, b), ('Train', 'Test'), fontsize=fontsize, handletextpad=0.1, borderpad=0.1)
plt.rcParams['font.family'] = "Arial"
plt.tight_layout()
savefig("xgb.jpg", bbox_inches='tight')
np.savetxt('xgb_y_test.csv', y_test_hat, delimiter=',')
np.savetxt('xgb_y_train.csv', y_train_hat, delimiter=',')
np.savetxt('xgb_x_test.csv', y_test, delimiter=',')
np.savetxt('xgb_x_train.csv', y_train, delimiter=',')
plt.show()
print('r:', stats.pearsonr(y_train, y_train_hat))
print('r:', stats.pearsonr(y_test, y_test_hat))
print('XGB在训练集的r:', stats.pearsonr(y_train, y_train_hat))
print('XGB在测试集的r:', stats.pearsonr(y_test, y_test_hat))
print('XGB在训练集的决定系数R2: %.3f' % (r2_score(y_train, y_train_hat)))  # 计算均方误差回归损失，越接近于1拟合效果越好决定系数R平方
print('XGB在训练集的均方根误差RMSE: %.3f' % (np.sqrt(mean_squared_error(y_train, y_train_hat))))  # 计算均方差根判断效果RMSE（Root Mean Squared Error）
print('XGB在训练集的平均绝对误差MAE: %.3f' % (mean_absolute_error(y_train, y_train_hat)))  # 计算平均绝对误差 MAE
print('XGB在训练集的平均绝对百分误差MAPE: %.3f' % (mean_absolute_percentage_error(y_train, y_train_hat)))  # 计算平均绝对误差 MAPE
print('XGB在测试集的决定系数R2: %.3f' % (r2_score(y_test, y_test_hat)))  # 计算均方误差回归损失，越接近于1拟合效果越好决定系数R平方
print('XGB在测试集均方根误差RMSE: %.3f' % (np.sqrt(mean_squared_error(y_test, y_test_hat))))  # 计算均方差根判断效果RMSE（Root Mean Squared Error）
print('XGB在测试集的平均绝对误差MAE: %.3f' % (mean_absolute_error(y_test, y_test_hat)))  # 计算平均绝对误差 MAE
print('XGB在测试集的平均绝对百分误差MAPE: %.3f' % (mean_absolute_percentage_error(y_test, y_test_hat)))  # 计算平均绝对误差 MAPE
print('参数的最佳取值：{0}'.format(model.best_params_))

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

# 读取用户上传的原始数据文件
file_path = "./sample.csv"
data = pd.read_csv(file_path, encoding="gbk")

# 获取特征数
num_features = data.shape[1] - 1  # 目标变量 'Adsorption capacity' 不计入特征数
print(f"特征数: {num_features}")

# 分离特征和目标变量
X = data.drop(columns=['Adsorption capacity'])
y = data['Adsorption capacity']

# 数据标准化
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

# 训练XGBoost模型
best_params = {
    'learning_rate': 0.1,
    'max_depth': 3,
    'min_child_weight': 3,
    'colsample_bytree': 0.8,
    'gamma': 0.1,
    'reg_alpha': 0.1,
    'reg_lambda': 3
}

xgb_regressor = xgb.XGBRegressor(**best_params)
xgb_regressor.fit(X_std, y)

# 提取所有 "Substrate_", "Heavy metal ions_", 和 "Modified" 相关的列
substrate_columns = [col for col in data.columns if col.startswith("Substrate_")]
heavy_metal_columns = [col for col in data.columns if col.startswith("Heavy metal ions_")]
modified_columns = [col for col in data.columns if col.startswith("Modified")]

# 创建包含所有情况的默认输入值字典（默认设为0）
default_feature_values = {col: 0 for col in substrate_columns + heavy_metal_columns + modified_columns}

# 创建与模型输入格式匹配的 DataFrame（X_input）
def create_input_dataframe(feature_values):
    """
    创建输入数据框，保证与模型训练时的特征匹配
    :param feature_values: 字典，包含所有特征名称及其对应的值
    :return: DataFrame 格式的输入数据
    """
    X_input = pd.DataFrame([feature_values])
    return X_input

def predict_external_data(external_data):
    print("🚀 开始预测...")
    
    # 重新排序，使其特征顺序与训练时一致
    external_data = external_data.reindex(columns=scaler.feature_names_in_, fill_value=0)
    
    print("✅ 输入数据预处理完成，形状:", external_data.shape)

    # 尝试标准化数据
    try:
        external_data_std = scaler.transform(external_data)
        print("✅ 标准化完成")
    except Exception as e:
        print(f"❌ Scaler transform 出错: {e}")
        return None  # 返回 None 避免后续错误

    # 尝试进行预测
    try:
        predictions = xgb_regressor.predict(external_data_std)
        print("✅ 预测完成，结果:", predictions)
    except Exception as e:
        print(f"❌ XGBoost 预测出错: {e}")
        return None  # 返回 None 避免后续错误

    return predictions



# 示例：
# 定义一个样本的特征值（用户需要提供实际值）
feature_values = {
    'Specific surface area': 100.0,
    'Pore volume': 0.2,
    'Average pore': 5.0,
    'Dosage': 0.5,
    'Initial concentration': 10.0,
    'Temperature': 25,
    'Contact time': 30,
    'pH': 6.0,
    'Substrate_Fe-Mn binary compound': 0,
    'Substrate_MnO2': 1,
    'Substrate_MnOx': 0,
    'Heavy metal ions_As(Ⅲ)': 0,
    'Heavy metal ions_As(Ⅴ)': 0,
    'Heavy metal ions_Cd(Ⅱ)': 0,
    'Heavy metal ions_Co(Ⅱ)': 0,
    'Heavy metal ions_Cr(Ⅲ)': 0,
    'Heavy metal ions_Cr(Ⅵ)': 1,
    'Heavy metal ions_Cu(Ⅱ)': 0,
    'Heavy metal ions_Fe(Ⅱ)': 0,
    'Heavy metal ions_Hg(Ⅱ)': 0,
    'Heavy metal ions_Mn(Ⅱ)': 0,
    'Heavy metal ions_Ni(Ⅱ)': 0,
    'Heavy metal ions_Pb(Ⅱ)': 0,
    'Heavy metal ions_Sb(Ⅲ)': 0,
    'Heavy metal ions_Sb(Ⅴ)': 0,
    'Heavy metal ions_Zn(Ⅱ)': 0,
    'Modified_Chemical modification': 0,
    'Modified_Composite modification': 0,
    'Modified_Magnetic properties': 0,
    'Modified_Nano modification': 0,
    'Modified_Others': 1
}

# 创建输入数据
X_input = create_input_dataframe(feature_values)

# 调用预测函数
y_pred = predict_external_data(X_input)

# 只有 y_pred 不是 None 时才打印
if y_pred is not None:
    print(f"Predicted Adsorption Capacity: {y_pred[0]:.4f} mg/g")
else:
    print("❌ 预测失败，请检查错误信息！")



# 输出预测结果
#print(f"Predicted Adsorption Capacity: {y_pred[0]:.4f}")


特征数: 31
🚀 开始预测...
✅ 输入数据预处理完成，形状: (1, 31)
✅ 标准化完成
✅ 预测完成，结果: [28.7449]
Predicted Adsorption Capacity: 28.7449 mg/g
